# APIs Lab
In this lab we will practice using APIs to retrieve and store data.

In [1]:
# Imports at the top
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt
%matplotlib inline

## Exercise 1: Get Data From Sheetsu

[Sheetsu](https://sheetsu.com/) is an online service that allows you to access any Google spreadsheet from an API. This can be a very handy way to share a dataset with colleagues as well as to create a mini centralized data storage, that is simpler to edit than a database.

A Google Spreadsheet with wine data can be found [here](https://docs.google.com/a/generalassemb.ly/spreadsheets/d/1JWRwDnwIMLgvPqNMdJLmAJgzvz0K3zAUc6jev3ci1c8/edit?usp=sharing).

You can access it through the Sheetsu API at this endpoint: https://sheetsu.com/apis/v1.0/cc9420722ae4. [Here](https://sheetsu.com/docs/beta) is Sheetsu's documentation.


Questions:

1. Use the requests library to access the document. Inspect the response text. What kind of data is it?
2. Check the status code of the response object. What code is it?
3. Use the appropriate libraries and read functions to read the response into a Pandas Dataframe
4. Once you've imported the data into a dataframe, check the value of the 5th line: what's the price?

In [2]:
api_url = 'https://sheetsu.com/apis/v1.0/cc9420722ae4'

In [3]:
api_response = requests.get(api_url)

In [4]:
api_response

<Response [200]>

In [5]:
api_response.text[:200]

u'[{"Color":"W","Region":"Portugal","Country":"Portugal","Vintage":"2013","Vinyard":"Vinho Verde","Name":"","Grape":"","Consumed In":"2015","Score":"4","Price":""},{"Color":"W","Region":"France","Countr'

In [6]:
response = json.loads(api_response.text)

In [7]:
response[5]

{u'Color': u'R',
 u'Consumed In': u'2015',
 u'Country': u'US',
 u'Grape': u'cab',
 u'Name': u'',
 u'Price': u'13',
 u'Region': u'California',
 u'Score': u'3.5',
 u'Vintage': u'2012',
 u'Vinyard': u'Crow Canyon'}

In [8]:
type(response)

list

In [9]:
response[0]

{u'Color': u'W',
 u'Consumed In': u'2015',
 u'Country': u'Portugal',
 u'Grape': u'',
 u'Name': u'',
 u'Price': u'',
 u'Region': u'Portugal',
 u'Score': u'4',
 u'Vintage': u'2013',
 u'Vinyard': u'Vinho Verde'}

In [10]:
api_response.status_code

200

In [11]:
wine = pd.DataFrame(response)
wine.head()

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3,2013,Peyruchet
2,W,2015,Oregon,,,20,Oregon,3,2013,Abacela
3,W,2015,Spain,chardonay,,7,Spain,2.5,2012,Ochoa
4,R,2015,US,"chiraz, cab",Spice Trader,6,,3,2012,Heartland


In [12]:
wine = pd.read_json(api_response.text)
wine.tail(2)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
49,R,2015,US,,My wonderful wine,200,Sonoma,10,1973,
50,R,2015,US,,My wonderful wine,200,Sonoma,10,1973,


> Answers:
    1. A JSON string.
    2. 200
    3. Options inlucde: pd.read_json; json.loads + pd.Dataframe
    4. 5

### Exercise 2: Post Data to Sheetsu
Now that we've learned how to read data, it'd be great if we could also write data. For this we will need to use a _POST_ request.

1. Use the post command to add the following data to the spreadsheet:

In [13]:
post_data = {
'Grape' : ''
, 'Name' : 'My wonderful wine'
, 'Color' : 'R'
, 'Country' : 'US'
, 'Region' : 'Sonoma'
, 'Vinyard' : ''
, 'Score' : '10'
, 'Consumed In' : '2015'
, 'Vintage' : '1973'
, 'Price' : '200'
}

1. What status did you get? How can you check that you actually added the data correctly?
- In this exercise, your classmates are adding data to the same spreadsheet. What happens because of this? Is it a problem? How could you mitigate it?

In [14]:
requests.post(api_url, data=post_data)

<Response [201]>

## Exercise 3: Data munging

Get back to the dataframe you've created in the beginning. Let's do some data munging:

1. Search for missing data
    - Is there any missing data? How do you deal with it?
    - Is there any data you can just remove?
    - Are the data types appropriate?
- Summarize the data 
    - Try using describe, min, max, mean, var

In [15]:
wine.head(15)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3,2013,Peyruchet
2,W,2015,Oregon,,,20,Oregon,3,2013,Abacela
3,W,2015,Spain,chardonay,,7,Spain,2.5,2012,Ochoa
4,R,2015,US,"chiraz, cab",Spice Trader,6,,3,2012,Heartland
5,R,2015,US,cab,,13,California,3.5,2012,Crow Canyon
6,R,2015,US,,#14,21,Oregon,2.5,2013,Abacela
7,R,2015,France,"merlot, cab",,12,Bordeaux,3.5,2012,David Beaulieu
8,R,2015,France,"merlot, cab",,11.99,Medoc,3.5,2011,Chantemerle
9,R,2015,US,merlot,,13,Washington,4,2011,Hyatt


In [16]:
wine.describe()

,Consumed In,Vintage
count,51.000000,51.000000
mean,2014.725490,1995.254902
std,0.602609,19.585549
min,2013.000000,1973.000000
25%,2015.000000,1973.000000
50%,2015.000000,2011.000000
75%,2015.000000,2012.000000
max,2015.000000,2013.000000


In [41]:
wine = wine.replace('', np.nan)

In [42]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 10 columns):
Color          51 non-null object
Consumed In    51 non-null int64
Country        49 non-null object
Grape          18 non-null object
Name           42 non-null object
Price          45 non-null float64
Region         50 non-null object
Score          50 non-null float64
Vintage        51 non-null int64
Vinyard        29 non-null object
dtypes: float64(2), int64(2), object(6)
memory usage: 4.4+ KB


In [43]:
wine[['Score', 'Price']] = wine[['Score', 'Price']].astype(float)

In [46]:
wine.head(15)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,NaN,NaN,NaN,Portugal,4.0,2013,Vinho Verde
1,W,2015,France,NaN,NaN,17.80,France,3.0,2013,Peyruchet
2,W,2015,Oregon,NaN,NaN,20.00,Oregon,3.0,2013,Abacela
3,W,2015,Spain,chardonay,NaN,7.00,Spain,2.5,2012,Ochoa
4,R,2015,US,"chiraz, cab",Spice Trader,6.00,NaN,3.0,2012,Heartland
5,R,2015,US,cab,NaN,13.00,California,3.5,2012,Crow Canyon
6,R,2015,US,NaN,#14,21.00,Oregon,2.5,2013,Abacela
7,R,2015,France,"merlot, cab",NaN,12.00,Bordeaux,3.5,2012,David Beaulieu
8,R,2015,France,"merlot, cab",NaN,11.99,Medoc,3.5,2011,Chantemerle
9,R,2015,US,merlot,NaN,13.00,Washington,4.0,2011,Hyatt


## Exercise 4: Feature Extraction

We would like to use a regression tree to predict the score of a wine. In order to do that, we first need to select and engineer appropriate features.

- Set the target to be the Score column, drop the rows with no score
- Use pd.get_dummies to create dummy features for all the text columns
- Fill the nan values in the numerical columns, using an appropriate method
- Train a Decision tree regressor on the Score, using a train test split:
        X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.3, random_state=42)
- Plot the test values, the predicted values and the residuals
- Calculate R^2 score
- Discuss your findings


In [49]:
dropped = wine[wine['Score'].isnull()]

In [50]:
y = dropped['Score']

In [51]:
numerical = ['Consumed In', 'Price', 'Vintage']

In [52]:
dummies = [c for c in dropped.columns if c not in numerical + ['Score']]

In [53]:
X = dropped[numerical].join(pd.get_dummies(dropped[dummies]))

## Exercise 5: IMDB Movies

Sometimes an API doesn't provide all the information we would like to get and we need to be creative.
Here we will use a combination of scraping and API calls to investigate the ratings and gross earnings of famous movies.

## 5.a Get top movies

The Internet Movie Database contains data about movies. Unfortunately it does not have a public API.

The page http://www.imdb.com/chart/top contains the list of the top 250 movies of all times. Retrieve the page using the requests library and then parse the html to obtain a list of the `movie_ids` for these movies. You can parse it with regular expression or using a library like `BeautifulSoup`.

**Hint:** movie_ids look like this: `tt2582802`

## 5.b Get top movies data

Although the Internet Movie Database does not have a public API, an open API exists at http://www.omdbapi.com.

Use this API to retrieve information about each of the 250 movies you have extracted in the previous step.
- Check the documentation of omdbapi.com to learn how to request movie data by id
- Define a function that returns a python object with all the information for a given id
- Iterate on all the IDs and store the results in a list of such objects
- Create a Pandas Dataframe from the list

## 5.c Get gross data

The OMDB API is great, but it does not provide information about Gross Revenue of the movie. We'll revert back to scraping for this.

- Write a function that retrieves the gross revenue from the entry page at imdb.com
- The function should handle the exception of when the page doesn't report gross revenue
- Retrieve the gross revenue for each movie and store it in a separate dataframe

## 5.d Data munging

- Now that you have movie information and gross revenue information, let's clean the two datasets.
- Check if there are null values. Be careful they may appear to be valid strings.
- Convert the columns to the appropriate formats. In particular handle:
    - Released
    - Runtime
    - year
    - imdbRating
    - imdbVotes
- Merge the data from the two datasets into a single one

## 5.d Text vectorization

There are several columns in the data that contain a comma separated list of items, for example the Genre column and the Actors column. Let's transform those to binary columns using the count vectorizer from scikit learn.

Append these columns to the merged dataframe.

**Hint:** In order to get the actors name right, you'll have to set the `token_pattern` parameter in `CountVectorizer` to u'(?u)\\w+\.?\\w?\.? \\w+'. Can you see why? How does this differ from the default?

## Bonus:

- What are the top 10 grossing movies?
- Who are the 10 actors that appear in the most movies?
- What's the average grossing of the movies in which each of these actors appear?
- What genre is the oldest movie?
